In [1]:
import numpy as np
import pandas as pd
import requests
import os

In [4]:
Season_All_22_23 = pd.read_csv('E0.csv')
Season_All_22_23.head(10)
print(Season_All_22_23.info())
Season_All_22_23.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Columns: 106 entries, Div to AvgCAHA
dtypes: float64(82), int64(16), object(8)
memory usage: 314.8+ KB
None


,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,HF,AF,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
count,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,...,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.634211,1.218421,0.757895,0.563158,13.952632,11.310526,4.907895,3.894737,10.597368,10.931579,...,2.100605,-0.281579,1.953447,1.948737,1.971184,1.957632,2.019289,2.010605,1.942763,1.933447
std,1.419944,1.183518,0.918480,0.746998,5.604170,4.941173,2.495260,2.230627,3.288020,3.463424,...,0.396997,0.874546,0.100258,0.097179,0.100997,0.098838,0.103461,0.101557,0.092671,0.090241
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000,3.000000,...,1.470000,-2.750000,1.650000,1.580000,1.710000,1.610000,1.810000,1.650000,1.760000,1.600000
25%,1.000000,0.000000,0.000000,0.000000,10.000000,8.000000,3.000000,2.000000,8.000000,8.000000,...,1.837500,-0.750000,1.880000,1.870000,1.880000,1.880000,1.930000,1.930000,1.860000,1.860000
50%,1.000000,1.000000,1.000000,0.000000,14.000000,11.000000,5.000000,4.000000,10.000000,11.000000,...,1.980000,-0.250000,1.960000,1.950000,1.960000,1.950000,2.010000,2.010000,1.940000,1.930000
75%,2.000000,2.000000,1.000000,1.000000,17.000000,15.000000,7.000000,5.000000,13.000000,13.000000,...,2.290000,0.250000,2.040000,2.030000,2.050000,2.040000,2.100000,2.090000,2.020000,2.010000
max,9.000000,6.000000,5.000000,3.000000,33.000000,30.000000,15.000000,11.000000,23.000000,24.000000,...,4.060000,1.750000,2.350000,2.200000,2.480000,2.270000,2.520000,2.290000,2.420000,2.120000


In [5]:

#loading in all additional .csv files
additional_files = ['B1.csv', 'D1.csv', 'D2.csv', 'E1.csv', 'E2.csv', 'F1.csv', 'F2.csv', 'G1.csv', 'I1.csv',
                        'I2.csv', 'N1.csv', 'P1.csv', 'SC0.csv', 'SP1.csv', 'SP2.csv', 'T1.csv']
additional_dfs = [pd.read_csv(file) for file in additional_files]


In [6]:

#merging additional dfs into single dataframe
Season_All_22_23 = pd.concat([Season_All_22_23] + additional_dfs, ignore_index = True)
print(Season_All_22_23.info())
Season_All_22_23.head(1)

#defining what columns are needed for simplified model
selected_columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
Scores_Results_22_23 = Season_All_22_23[selected_columns]
Scores_Results_22_23.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6185 entries, 0 to 6184
Columns: 106 entries, Div to AvgCAHA
dtypes: float64(95), int64(2), object(9)
memory usage: 5.0+ MB
None


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,05/08/2022,Crystal Palace,Arsenal,0,2,A
1,06/08/2022,Fulham,Liverpool,2,2,D
2,06/08/2022,Bournemouth,Aston Villa,2,0,H
3,06/08/2022,Leeds,Wolves,2,1,H
4,06/08/2022,Newcastle,Nott'm Forest,2,0,H


In [7]:

#recoding how full-time result is represented
recoding_dict = {'H':1, 'D':0.5, 'A':0}
Scores_Results_22_23['FTR'] = Scores_Results_22_23['FTR'].replace(recoding_dict)
Scores_Results_22_23.head(5)

#creating a list of club names
club_names = set(Scores_Results_22_23['HomeTeam'])
for club in club_names:
    print(club)

#populating ELO table with all clubs
club_names_list = list(club_names)
ClubELO = pd.DataFrame({
    'Club': club_names_list,
    'ELO': 1000
})

print(ClubELO)

# Defining ELO constants
k = 40         # ELO factor, can be adjusted
home_adv = 30  # Additional ELO points for th

Celta
Fulham
Aberdeen
Utrecht
Ein Frankfurt
Palermo
Sp Braga
Malaga
Ascoli
Oxford
Milton Keynes Dons
Chelsea
Paris FC
Morecambe
Cambridge
Wigan
Frosinone
Spezia
Go Ahead Eagles
Ionikos
Metz
Villarreal B
Fenerbahce
Benevento
St Pauli
Antalyaspor
Blackburn
Ipswich
PSV Eindhoven
QPR
Crystal Palace
AEK
Ajaccio
Sheffield Weds
Verona
Lille
Brescia
Ajax
Ankaragucu
OFI Crete
Eibar
Zaragoza
Albacete
Sociedad
Charlton
Anderlecht
Como
Salernitana
Hansa Rostock
Middlesbrough
Cambuur
Wolfsburg
Volos NFC
Quevilly Rouen
Amiens
Nijmegen
Napoli
Chaves
Lazio
Greuther Furth
Vallecano
Regensburg
Caen
Bordeaux
Brest
Bari
Paderborn
Sivasspor
Cardiff
Schalke 04
Udinese
Cagliari
Reggina
Portimonense
PAOK
Sp Lisbon
Swansea
Leeds
Sassuolo
Tottenham
Paris SG
Volendam
Trabzonspor
Oud-Heverlee Leuven
Monaco
Cremonese
Levante
West Ham
Kasimpasa
Holstein Kiel
Sheffield United
Atalanta
Man City
Hibernian
Valenciennes
Sunderland
Cosenza
Arouca
Seraing
Burton
Umraniyespor
Getafe
Huesca
Guingamp
Stoke
Konyaspor
Karagumr

C:\Users\n529634\AppData\Local\Temp\ipykernel_90212\2243063522.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Scores_Results_22_23['FTR'] = Scores_Results_22_23['FTR'].replace(recoding_dict)
C:\Users\n529634\AppData\Local\Temp\ipykernel_90212\2243063522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Scores_Results_22_23['FTR'] = Scores_Results_22_23['FTR'].replace(recoding_dict)


In [ ]:
def calculate_elo_update(home_elo, away_elo, actual_home_score, k_factor, home_adv): 
    # Adjusting home ELO for home advantage
    home_elo_adjusted = home_elo + home_adv
    
    # Calculating expected scores
    expected_home_score = 1 / (1 + 10 ** ((away_elo - home_elo_adjusted) / 400))
    expected_away_score = 1 - expected_home_score
    
    # Calculate new Elo ratings based on actual outcome
    new_home_elo = home_elo + k_factor * (actual_home_score - expected_home_score)
    new_away_elo = away_elo + k_factor * ((1 - actual_home_score) - expected_away_score)
    
    return new_home_elo, new_away_elo

# Sort the DataFrame by date in ascending order
Scores_Results_22_23 = Scores_Results_22_23.sort_values(by='Date').reset_index(drop=True)

In [ ]:
# Iterate over each match in the sorted Scores_Results_22_23 DataFrame
for _, match in Scores_Results_22_23.iterrows():
    home_team = match['HomeTeam']
    away_team = match['AwayTeam']
    outcome = match['FTR']  # 'FTR' gives the actual score for the home team: 1.0 (home win), 0.5 (draw), 0.0 (away win)

    # Retrieve current Elo ratings for both teams
    home_elo = ClubELO.loc[ClubELO['Club'] == home_team, 'ELO'].values[0]
    away_elo = ClubELO.loc[ClubELO['Club'] == away_team, 'ELO'].values[0]
    
    # Calculate new Elo ratings
    new_home_elo, new_away_elo = calculate_elo_update(home_elo, away_elo, outcome, k, home_adv)
    
    # Update the ClubELO DataFrame with the new ratings
    ClubELO.loc[ClubELO['Club'] == home_team, 'ELO'] = new_home_elo
    ClubELO.loc[ClubELO['Club'] == away_team, 'ELO'] = new_away_elo

# Display updated ClubELO DataFrame
print(ClubELO.sort_values(by='ELO'))

C:\Users\n529634\AppData\Local\Temp\ipykernel_90212\1648107258.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1018.2773403191115' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ClubELO.loc[ClubELO['Club'] == home_team, 'ELO'] = new_home_elo


              Club          ELO
0            Celta   924.798118
1           Fulham  1021.378108
2         Aberdeen   993.667552
3          Utrecht  1051.793064
4    Ein Frankfurt  1024.756624
..             ...          ...
320      Marseille  1113.352788
321      Darmstadt  1137.782914
322           Nice  1065.980578
323   Bristol City   977.270461
324     Livingston   924.693667

[325 rows x 2 columns]


In [ ]:
ranked = ClubELO.sort_values(by='ELO', ascending=False)

,Club,ELO
282,Benfica,1242.926177
148,Celtic,1235.415094
186,Plymouth,1223.921374
56,Napoli,1222.966545
154,Rangers,1220.849647
...,...,...
249,Hatayspor,818.374623
50,Cambuur,815.232347
196,Forest Green,810.316775
98,Seraing,803.068469
